# Neural networks for text classification

This demo provides very basic Numpy implementations of the following:

1. A feed forward NN represented as a doubly linked list
2. A text classifier that uses a CBOW feature representation
3. Autograd

*Note: This demo conveys some of the underlying constructs used in deep learning packages, not their implementations of those constructs.*

### Load DBPedia14 dataset

In [1]:
import datasets
import numpy as np
import pandas as pd

M = 2500

df = datasets.load_dataset(
    'dbpedia_14', 
    split=['train[:100%]', 
           'test[100%:]']
)[0].to_pandas().sample(frac=1).reset_index(drop=True)[:M]

K = len(set(df.label))

Reusing dataset d_bpedia14 (/Users/chris/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e)


### Spacy preprocessing pipeline

In [2]:
import re
from tqdm import tqdm
from typing import List

import spacy
from spacy.language import Language

pipeline_name = 'DBPedia14'


def camel_case_split(str):
    return " ".join([wrd for wrd in re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)])


@Language.component(pipeline_name)
def preprocess(doc):
    doc = [token for token in doc if not token.is_punct]
    # doc = [token for token in doc if not token.is_stop]
    doc = [token.text.lower().strip() for token in doc]
    doc = [token for token in doc if 0 < len(token) <= 12]
    return " ".join(doc)


class Pipeline:
    
    # http://emailregex.com/
    email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)
    *|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]
    |\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9]
    (?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}
    (?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:
    (?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""
    # replace = [ (pattern-to-replace, replacement),  ...]
    replace = [
        ("<[^>]*>", " "),
        (email_re, " "),                           # Matches emails
        (r"(?<=\d),(?=\d)", ""),                   # Remove commas in numbers
        (r"\d+", " "),                             # Map digits to special token <numbr>
        (r"[*\^\.$&@<>,\-/+{|}=?#:;'\"\[\]]", ""), # Punctuation and other junk
        (r"[\n\t\r]", " "),                        # Removes newlines, tabs, creturn
        (r"[^\x00-\x7F]+", ""),                    # Removes non-ascii chars
        (r"\\+", " "),                             # Removes double-backslashs
        (r"\s+n\s+", " "),                         # 'n' leftover from \\n
        (r"\s+", " ")                              # Strips extra whitespace
    ]
    
    def __init__(self):
        self.pipeline = spacy.load('en_core_web_sm')
        self.pipeline.add_pipe(pipeline_name);
        
    def __call__(self, *args, **kwargs):
        return self.transform(*args, **kwargs)

    def transform(self, doc: str):
        for repl in self.replace:
            doc = re.sub(repl[0], repl[1], doc)
        doc = camel_case_split(doc)
        return self.pipeline(doc)
    
pipeline = Pipeline();

### Process data

In [3]:
from tqdm import tqdm

with tqdm(total=M) as bar:
    for i, content in enumerate(df.content.tolist()):
        df.content[i] = pipeline(content)
        bar.update(1)

  0%|                                                                                                                                                                                    | 0/2500 [00:00<?, ?it/s]/Users/chris/Documents/GU/GU-ANLY-580/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:15<00:00, 163.64it/s]


### Featurizer

Below we will build a BOW featurizer, and use it as both a CBOW featurizer and as an embedding lookup

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vocab_size = len(set(" ".join(df.content.tolist()).split(" ")))

featurizer = CountVectorizer(max_features=vocab_size, stop_words=None)
featurizer.fit(df.content.tolist());
featurizer.get_idx = {word: idx for idx, word in enumerate(featurizer.get_feature_names())}
featurizer.get_word = {idx: word for idx, word in enumerate(featurizer.get_feature_names())}

N = len(featurizer.get_idx)
N

11697

### Features

In [5]:
# Labels
Y = df.label.to_numpy()

# One-hot features (just word index pointers)
X = [[featurizer.get_idx[word] for word in doc.split(" ") 
            if featurizer.get_idx.get(word) is not None] for doc in df.content]

# BOW features
X_bow = np.zeros(shape=[M, N], dtype=int)
for i, seq in enumerate(X):
    for idx in seq:
        X_bow[i, idx] += 1

In [6]:
[featurizer.get_word[idx] for idx, c in enumerate(X_bow[0]) if c]

['jazz',
 'joe',
 'jones',
 'lewis',
 'mel',
 'orchestra',
 'presenting',
 'records',
 'solid',
 'state',
 'thad',
 'williams']

## Neural network as a graph

Deep learning packages implement neural networks as a graph, where each node represents a layer within the network, and each edge represents a transformation that maps one layer onto the another. The two most important components of any deep learning library are (i) efficient implementations of the various transformations used in deep learning (graph edges) that are optimized for various hardware (x86, ARM, NVIDIA GPUS, TPUs etc..), and (ii) automatic differentiation (autograd) which enables users to use the package constructs (linear transformations, activations, loss functions, optimizers etc.) without ever having to implement (or even think about) backpropogation. Below is a (very rudimentary and fairly inefficient) implementation of the basic computational units that are needed to implement a feed forward network.

*Note: There are two numpy multiplication operations being used here: 1) `@` for matrix multiplication, and 2) `*` for the element-wise (aka Hadamard, aka Schur) product*

### Cross entropy loss

In [7]:
def cross_entropy(P, y):
    ce = np.sum(-y * np.log(P), axis=1)
    return np.mean(ce)

### Metrics

In [8]:
from sklearn.metrics import accuracy_score

def compute_metrics(P, Y):
    Y_hat = np.argmax(P, axis=1)
    acc = accuracy_score(Y, Y_hat)
    return acc

### Softmax

In [9]:
def softmax(Z):
    Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    partition = np.sum(Z, axis=1, keepdims=True)
    return Z / partition

### Weight initializer

Weight initialization is crucially important in deep learning, if not done properly your network will just diverge after the first gradient update. There are many options here, but it primarily boils down to what distribution do you want your intial weight values to be drawn from. A great choice is *Xavier* initialization, named after it's author Xavier Glorot, which draws weights from a normal distribution $W_{ij} \sim N(\mu=0, \sigma^2=n)$, where $W \in \mathbb{R}^{n \times \cdot}$. For large weight matrices, this is a means to control the variance of the output, $z$ at each layer.

In [10]:
def weight_init(*dims):
    """ Xavier initialization """
    return (1 / np.sqrt(dims[0])) * np.random.randn(*dims)

### Dropout module

Dropout is one of the most important regularization techniques in deep learning. In each activation layer during training, a random subset (chosen with probability `prob`) of the activations are masked out (zeroed). On an intuitive and somewhat handwavy level, you can think of this as a way to prevent the network over relying on on a small subset of the pathways through the network, and helps force all of the nodes, and therefore weights, to be utilized (quasi) equally. By doing this, we are scaling the values of the output layer; in order to keep the distribution over each layer consistent during the training and test phases, this means we must apply a scaling factor either during test time ($\frac{1}{prob}$, traditional dropout), or during training ($\frac{1}{1-prob}$, inverted dropout). Inverted dropout is the preferred method as it incurs no runtime cost during inference, while only adding marginal runtime overhead during training (remember, the vast majority of the computation during training is from backpropogation). This is an implementation of inverted dropout.

In [11]:
def apply_dropout(a: np.ndarray, prob: float):
    mask = np.random.choice([0.0, 1.0], size=a.shape, p=[prob, 1 - prob])
    a *= mask / (1 - prob)
    return a

### Graph node

The doubly linked list is a natural data structure for implementing networks. Below we make a class called `Node` which will serve as the base class for the various layer types in our network. In this implementation, each `Node` will contain three things: (i) memory to store a layer (`Node.out`), (ii) a method to map the parent node's output layer to its output (`Node.out = Node.forward(Node.last.out)`), (iii) a method to backpropogate gradients from its child node to its parent node (`Node.error = Node.backward(Node.next.error)`), and (iv) a set of methods to perform gradient updates to the parameters that it owns (`Node.update()`, `Node.zero_grad()`).

In [12]:
class Node:
    
    """ 
    Node in computational graph (doubly linked list)
    Note: not to be confused with a node in a network layer
    
    Attributes:
    last: Node        # Parent
    next: Node        # Child
    out: np.ndarray   # Output layer
    error: np.ndarray # dCE/d`out`
    dim: int          # layer dimension
    lr: float         # learning rate
    """
    
    def __init__(self, last, dim: int, lr: float = None):
        self.last = last
        if self.last:
            self.last.next = self
        self.next = None
        self.out = None
        self.error = None
        self.dim = dim
        self.lr = lr
        
    def forward(self, dropout: bool = False):
        pass
    
    def backward(self):
        pass
    
    def zero_grad(self):
        pass
    
    def update(self):
        pass

### Input embedding node

This is an embedding lookup, which will be the first layer in our network. The forward method accepts a batch of BOW features and computes their inner-product with an embedding lookup table. This is an implementation of the continuous *bag-of-features*, in which we we simply sum the embedding for each word in the the BOW input, weighted by their frequency counts.

In [13]:
class InputEmbedding(Node):
    
    def __init__(self, vocab_size, embedding_dim, lr, wt_decay):
        super().__init__(None, embedding_dim, lr)
        self.W = weight_init(embedding_dim, vocab_size)
        self.b = weight_init(1, self.dim)
        self.wt_decay = wt_decay
    
    def forward(self, X, dropout):
        self.input = X
        self.out = X @ self.W.T + self.b
        if self.next:
            self.next.forward(dropout)
    
    def zero_grad(self):
        self.W_grad = np.zeros_like(self.W)
        self.b_grad = np.zeros_like(self.b)
    
    def update(self):
        n = self.out.shape[0]
        l1_grad = self.wt_decay * np.sign(self.W)
        l2_grad = self.wt_decay * self.W
        self.W -= self.lr * ((1 / n) * self.W_grad + l1_grad + l2_grad)
        self.b -= self.lr * (1 / n) * self.b_grad
    
    def backward(self, error=None):
        if error is None:
            error = self.next.error
        dW = self.input
        db = 1
        self.W_grad = error.T @ dW
        self.b_grad = np.sum(error, axis=0) * db

### Leaky relu activation

The leaky relu (rectified linear unit) is a popular activation function (non-linearity).

$LReLU(z) = \max(z, \alpha z) \quad \text{where} \quad \alpha \in (0, 1)$

In [14]:
class LReLU(Node):
    
    def __init__(self, last, alpha=0.001, dropout=0.5):
        super().__init__(last, last.dim)
        self.alpha = alpha
        self.dropout = dropout
    
    def forward(self, dropout):
        self.out = np.where(self.last.out >= 0, 
                            self.last.out, 
                            self.alpha * self.last.out)
        if dropout:
            self.out = apply_dropout(self.out, prob=self.dropout)
        self.next.forward(dropout)
        
    def update(self):
        self.last.update()
        
    def zero_grad(self):
        self.last.zero_grad()
        
    def backward(self):
        da = np.where(self.last.out >= 0, 1, self.alpha)
        self.error = self.next.error * da
        self.last.backward()

### Note on the *forward* and *backward* pass

In the above cell you'll note that the `.forward()` method contains a call to `Node.next.forward()`; this is what allows data to flow from input layer to output layer. Also notice that the `.backward()`, `.update()`, and `.zero_grad()` methods all contain calls to either/both its child (`Node.next`) or parent (`Node.last`) along with code that computes gradents and updates parameter weights; this is very basic implementation of *autograd*. These features allow us to simply call `head_node.forward(some_input)` to compute the *forward pass*, and `tail_node.backward()` to compute the *backward pass* in the training loop at the bottom of this notebook.

### Linear node

This is the *linear* unit, sometimes referred to as a `dense` layer. It's just a batched implementation of $XW^T + b$.

In [15]:
class Linear(Node):
    
    def __init__(self, last, dim: int, lr: float, wt_decay: float):
        super().__init__(last, dim, lr)
        self.W = weight_init(self.dim, self.last.dim)
        self.b = weight_init(1, self.dim)
        self.wt_decay = wt_decay

    def zero_grad(self):
        self.W_grad = np.zeros_like(self.W)
        self.b_grad = np.zeros_like(self.b)
        self.error = None
        self.last.zero_grad()
    
    def update(self):
        n = self.out.shape[0]
        l2_grad = self.wt_decay * self.W
        self.W -= self.lr * ((1 / n) * self.W_grad + l2_grad)
        self.b -= self.lr * (1 / n) * self.b_grad
        self.last.update()
    
    def forward(self, dropout):
        self.out = self.last.out @ self.W.T + self.b
        if self.next:
            self.next.forward(dropout=dropout)
        
    def backward(self, error=None):
        if error is None:
            error = self.next.error
        dW = self.last.out
        db = 1
        self.W_grad = error.T @ dW
        self.b_grad = np.sum(error, axis=0) * db
        self.error = error @ self.W
        self.last.backward()

## Network wrapper

This is a wrapper around our doubly linked list, it allows us to define one object for interacting with the network rather than having to keep track of the *head* and *tail* nodes of our network.

In [16]:
class Network:
    
    input: Node
    last: Node
    
    def set_lr(lr):
        node = self.input
        while node is not None:
            node.lr = lr
            node = node.next
            
    @property
    def lr(self):
        return self.last.lr
        
    @property
    def out(self):
        return self.last.out
    
    def forward(self, x, dropout=False):
        self.input.forward(x, dropout=dropout)
    
    def backward(self, loss):
        self.last.backward(loss)
        
    def update(self):
        self.last.update()
        
    def zero_grad(self):
        self.last.zero_grad()
        
        
class ANN(Network):
    def __init__(self, vocab_size, layer_size, output_dim, num_layers, lr, wt_decay, dropout):
        super().__init__()
        self.input = InputEmbedding(vocab_size, layer_size, lr, wt_decay)
        self.last = LReLU(self.input, dropout=dropout)
        for i in range(num_layers - 1):
            z = Linear(self.last, layer_size, lr, wt_decay)
            self.last = LReLU(z, dropout=dropout)
        self.last = Linear(self.last, output_dim, lr, wt_decay)

## Training loop

In [17]:
min_lr = 0.0005
wt_decay = 1e-7
dropout = 0.0
layer_size = 15
layers = 2
batch_size = 10
epochs = 250

In [18]:
net = ANN(N, layer_size, K, layers, lr=0.02, wt_decay=wt_decay, dropout=dropout)

one_hot_labels = np.eye(K)

shuffle_idx = np.arange(M)

with tqdm(total=epochs * (M // batch_size)) as bar:    
    
    for epoch in range(epochs):
        
        # Randomize data ordering
        np.random.shuffle(shuffle_idx)
        X_shfd = X_bow[shuffle_idx]
        Y_shfd = Y[shuffle_idx]
        
        for i in range(0, M, batch_size):
            
            # Get batch of x,y pairs
            x = X_shfd[i: i + batch_size]
            y = Y_shfd[i: i + batch_size]
            
            # Skip empty slices
            if not x.shape[0]:
                continue
            
            # Forward pass
            net.forward(x, dropout=True)
            
            # Compute softmax from net.out
            P = softmax(net.out)
            
            # Backward pass
            # Note: From lecture 05, remember that the derivative of the
            # cross entropy loss w.r.t. the input (z) to the softmax P(z)
            # is dCE/dz = P(z) - y.
            net.backward(P - one_hot_labels[y])
            
            # Apply gradient updates
            net.update()
            
            # Zero the stored gradients
            net.zero_grad()
            
            bar.update(1)
        
        # Learning rate annealing
        new_lr = max(min_lr, 0.9 * net.lr) 
        
        # Evaluate performance on training set w/o dropout
        net.forward(X_bow)
        P = softmax(net.out)
        ce_tr = cross_entropy(P, one_hot_labels[Y])
        acc_tr = compute_metrics(P, Y)
        
        bar.set_description("Epoch: %d, CE: %.4f, Acc: %.4f" % (epoch + 1, ce_tr, acc_tr))

Epoch: 250, CE: 0.0062, Acc: 0.9992: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62500/62500 [04:07<00:00, 252.78it/s]
